In [13]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import TensorBoard

In [14]:
DATA_ROOT = "MP_Data"
actions = np.array(['VolumeUp', 'VolumeDown', 'PlayPause', 'Idle'])
sequences = 30
frames = 30

action_map = {label:num for num, label in enumerate(actions)}

sequences_data = []
labels = []

In [15]:
for action in actions:
    for sequence in range(sequences):
        video_list = []

        for frame in range(frames):
            np_data = np.load(os.path.join(DATA_ROOT, action, str(sequence), f"{str(frame)}.npy"))
            video_list.append(np_data)

        video_list = np.array(video_list)
        velocity = np.diff(video_list, axis=0)
        zeros = np.zeros((1, 42))

        final_list = np.vstack([zeros, velocity])
        
        sequences_data.append(final_list)
        labels.append(action_map[action])

In [16]:
x = np.array(sequences_data)

y = to_categorical(labels).astype(int)

xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                                test_size=0.2,
                                                stratify=y)

print("First frame of data looks like:")
print(x[0][0])
print(f"Max value in data: {np.max(x)}")
print(f"Min value in data: {np.min(x)}")

First frame of data looks like:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Max value in data: 0.9700713753700256
Min value in data: -0.8957942724227905


In [17]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 42)))
model.add(Dropout(0.2))

model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

c:\Users\PC\Documents\Code\Python\CV\action-detector\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(xtrain, ytrain, epochs=200, callbacks=[TensorBoard(log_dir='Logs')])

model.save('action.h5')
print("Saved model")

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - categorical_accuracy: 0.1979 - loss: 1.3861
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - categorical_accuracy: 0.2604 - loss: 1.3817
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - categorical_accuracy: 0.2708 - loss: 1.3771
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - categorical_accuracy: 0.2708 - loss: 1.3675
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - categorical_accuracy: 0.3542 - loss: 1.3557
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.2708 - loss: 1.3311
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - categorical_accuracy: 0.3125 - loss: 1.2763
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.3542 - loss: 1.2339
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.6667 - loss: 1.1486
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 0.8229 - loss: 1.1295
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0

Saved model
